In [1]:
import pandas as pd
df = pd.read_excel('FlightDelays.xlsx', 'Data_for_Analysis')
df.shape

(2201, 13)

In [27]:
pip install pandas_ml

  Using cached pandas_ml-0.6.1-py3-none-any.whl (100 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
df.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight Status
0,1455,OH,1455,JFK,184,2004-01-01,5935,BWI,0,4,1,N940CA,ontime
1,1640,DH,1640,JFK,213,2004-01-01,6155,DCA,0,4,1,N405FJ,ontime
2,1245,DH,1245,LGA,229,2004-01-01,7208,IAD,0,4,1,N695BR,ontime
3,1715,DH,1709,LGA,229,2004-01-01,7215,IAD,0,4,1,N662BR,ontime
4,1039,DH,1035,LGA,229,2004-01-01,7792,IAD,0,4,1,N698BR,ontime


In [4]:
df.describe()

,CRS_DEP_TIME,DEP_TIME,DISTANCE,FL_NUM,Weather,DAY_WEEK,DAY_OF_MONTH
count,2201.000000,2201.000000,2201.000000,2201.000000,2201.000000,2201.000000,2201.000000
mean,1371.938664,1369.298955,211.871422,3815.086324,0.014539,3.905498,16.024989
std,432.697149,442.462754,13.316815,2409.750224,0.119725,1.903149,8.677390
min,600.000000,10.000000,169.000000,746.000000,0.000000,1.000000,1.000000
25%,1000.000000,1004.000000,213.000000,2156.000000,0.000000,2.000000,8.000000
50%,1455.000000,1450.000000,214.000000,2385.000000,0.000000,4.000000,16.000000
75%,1710.000000,1709.000000,214.000000,6155.000000,0.000000,5.000000,23.000000
max,2130.000000,2330.000000,229.000000,7924.000000,1.000000,7.000000,31.000000


In [5]:
#check the balance of the categories in dependent variable
#This would give us the naive rule
df['Flight Status'].value_counts()

ontime     1773
delayed     428
Name: Flight Status, dtype: int64

In [6]:
#calculate proportions of the above
df['Flight Status'].value_counts()/df['Flight Status'].count()

ontime     0.805543
delayed    0.194457
Name: Flight Status, dtype: float64

In [7]:
X = df.loc[:, ['CARRIER', 'DAY_WEEK', 'Weather',
               'DEST', 'ORIGIN', 'CRS_DEP_TIME']]
y = pd.Categorical(df['Flight Status'], 
                   categories=['ontime', 'delayed'])
y = y.codes

In [8]:
#bin departure time
from scipy import stats
dept_time = df['CRS_DEP_TIME']
dep_time_bins=[659, 759, 859, 959, 1059, 1159, 1259, 1359, 1459, 1559, 1659, 1759, 1859, 1959, 2059]

#arguments: X, values, bins
statistic, bin_edges, dep_time_bin_number = stats.binned_statistic(
    dept_time, dept_time, bins=dep_time_bins)
X['CRS_DEP_TIME'] = dep_time_bin_number

In [9]:
#get dummies for categorical variables
X = pd.get_dummies(X, columns=['CARRIER', 
            'DEST', 'ORIGIN', 'CRS_DEP_TIME', 'DAY_WEEK'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 38 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Weather          2201 non-null   int64
 1   CARRIER_CO       2201 non-null   uint8
 2   CARRIER_DH       2201 non-null   uint8
 3   CARRIER_DL       2201 non-null   uint8
 4   CARRIER_MQ       2201 non-null   uint8
 5   CARRIER_OH       2201 non-null   uint8
 6   CARRIER_RU       2201 non-null   uint8
 7   CARRIER_UA       2201 non-null   uint8
 8   CARRIER_US       2201 non-null   uint8
 9   DEST_EWR         2201 non-null   uint8
 10  DEST_JFK         2201 non-null   uint8
 11  DEST_LGA         2201 non-null   uint8
 12  ORIGIN_BWI       2201 non-null   uint8
 13  ORIGIN_DCA       2201 non-null   uint8
 14  ORIGIN_IAD       2201 non-null   uint8
 15  CRS_DEP_TIME_0   2201 non-null   uint8
 16  CRS_DEP_TIME_1   2201 non-null   uint8
 17  CRS_DEP_TIME_2   2201 non-null   uint8
 18  CRS_DEP_

In [10]:
df['ORIGIN'].value_counts()

DCA    1370
IAD     686
BWI     145
Name: ORIGIN, dtype: int64

In [11]:
df['CRS_DEP_TIME'].value_counts()

1455    138
1300    109
1900     99
700      92
2120     90
1500     77
900      77
1700     74
840      62
1715     61
1245     61
1830     58
630      57
1030     56
1430     52
1630     51
1730     50
1530     50
1100     48
1400     46
2100     45
1600     45
800      40
2030     31
1240     31
850      31
1645     30
1230     28
930      28
1710     28
1800     27
1720     27
1640     27
830      26
600      26
1359     25
730      24
1610     24
1000     23
1200     22
640      22
2000     22
645      21
1525     21
1130     20
1930     20
1330     19
735      17
1040     15
1039      6
1515      5
1315      4
845       3
925       3
2130      2
759       2
1605      1
1520      1
1725      1
Name: CRS_DEP_TIME, dtype: int64

In [12]:
df['DAY_WEEK'].value_counts()

5    391
4    372
3    320
1    308
2    307
7    253
6    250
Name: DAY_WEEK, dtype: int64

In [13]:
X = X.drop(columns=['CARRIER_DH', 'CRS_DEP_TIME_7',
                  'ORIGIN_DCA', 'DEST_JFK', 'DAY_WEEK_5'])

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                  test_size = .1,
                                                  random_state=20, 
                                                    stratify=y )

In [20]:
# run 10-fold cross validation as well as use a 10% hold out set

from sklearn.model_selection import cross_val_predict
from pandas_ml import ConfusionMatrix
clf = LogisticRegression(random_state=2, solver='liblinear')
y_pred = cross_val_predict(clf, X_train, y_train, cv=10)
cm = ConfusionMatrix(y_train, y_pred)
print(cm)
cm.print_stats()

NameError: name 'np' is not defined

In [ ]:
#now lets test on our test set
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
cm = ConfusionMatrix(y_test, y_pred)
print(cm)
cm.print_stats()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_scores = clf.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 1])#False positive rate and true positive rate values 
roc_auc = auc(fpr, tpr) # area under the curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--') #naive rule
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np
coefficients = np.column_stack((X.columns, np.reshape(clf.coef_, -1))) #join column names and coeffcients
print('Coefficients: \n', coefficients)